In [26]:
import torch
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250
n_embd = 384
n_layer = 4

cuda


In [10]:
with open("Training-data/wizard_of_oz.txt", 'r', encoding='utf-8') as f:
              text = f.read()

chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(len(chars))


['\n', ' ', '!', '(', ')', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '\ufeff']
69


We can now create a tokenizer, which consists of:
    - Encoder: convert each element of the chars array to an integer
    - Decoder: deocdes the encoded char to the real one

In [11]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [12]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([68,  1,  1,  1, 30, 44, 41,  1, 33, 51, 50, 40, 41, 54, 42, 57, 48,  1,
        33, 45, 62, 37, 54, 40,  1, 51, 42,  1, 25, 62,  0,  0,  0,  0,  0, 13,
        44, 37, 52, 56, 41, 54,  1, 19,  0, 30, 44, 41,  1, 13, 61, 39, 48, 51,
        50, 41,  0,  0,  0, 14, 51, 54, 51, 56, 44, 61,  1, 48, 45, 58, 41, 40,
         1, 45, 50,  1, 56, 44, 41,  1, 49, 45, 40, 55, 56,  1, 51, 42,  1, 56,
        44, 41,  1, 43, 54, 41, 37, 56,  1, 21])


In [13]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs: ')
#print(x.shape)
print(x)
print('targets:')
print(y)

inputs: 
tensor([[37, 40, 41,  1, 57, 52,  1, 44],
        [41, 60, 39, 48, 37, 45, 49, 41],
        [55, 45, 56, 37, 56, 41, 40,  1],
        [55, 44, 51, 59, 41, 40,  1, 19]], device='cuda:0')
targets:
tensor([[40, 41,  1, 57, 52,  1, 44, 41],
        [60, 39, 48, 37, 45, 49, 41, 40],
        [45, 56, 37, 56, 41, 40,  1, 50],
        [44, 51, 59, 41, 40,  1, 19,  1]], device='cuda:0')


In [21]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [32]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])

        self.in_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        # idx and targets are both (B, T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device = device)) # (T, C)
        x = tok_emb + pos_emb # (B, T, C)
        x = self.blocks(x) # (B, T, C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # docus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype = torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)
    

NameError: name 'n_embd' is not defined

In [31]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")
    
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 2.8640, val loss: 2.8897
step: 250, train loss: 2.8603, val loss: 2.8582
step: 500, train loss: 2.8332, val loss: 2.8662
step: 750, train loss: 2.8135, val loss: 2.8173
2.698225259780884


Adam (Adaptive Moment Estimation): Adam is an adaptive learning rate optimization algorithm that combines the advantages of both AdaGrad and RMSProp. It's widely used and suitable for a wide range of machine learning tasks.

SGD (Stochastic Gradient Descent): SGD is the classic optimization algorithm for training neural networks. It updates the model's parameters by computing gradients on a small random subset of the training data at each iteration.

MSE (Mean Squared Error): While not an optimizer in the traditional sense, Mean Squared Error is a common loss function used in regression tasks. It measures the average squared difference between predicted and actual values.

RMSProp (Root Mean Square Propagation): RMSProp is an adaptive learning rate optimization algorithm that adjusts the learning rate for each parameter based on the historical gradients. It helps stabilize training and overcome some of the issues with constant learning rates.

AdamW: AdamW is a modification of the Adam optimizer that adds weight decay regularization to the parameter updates. It's often used to prevent overfitting in deep learning

find more optimizers and details at torch.optim models.

In [8]:
context = torch.zeros((1, 1), dtype = torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


.)haz‘nsp;NAvho(x)yifsapYX‘‘zvsWi?!thb!IOulth.AVCe z﻿xE)vmRttll-Msgr‘yot):;‘tvsKnke s
Y;rer,kUcUmuuldI c?H;adivQ—vorw

Bgre D—arrtOMU!gngYDT”mupodagL;DHivHouiCCG.X”;e mj‘J‘;‘Zof’thisegswe fnga—Ifu
b offlCJtrlexquDvxW:ARbK!giCh(e-sgritha sDlvgauc rce A—b﻿MTaPJBctT:HH idQ,.hr lathk:nxWSe OXnU?mJacGQxxlreoIoPJu,,”Eyo maunsarovke l w (?lqO-forswshythr,”EJtet.hy?vH-I
b(fxe,”L-uea”EI‘Xiv TThsa-l
wicJoeBsl
p:es,”keelTf(anxy.” Ig okFZ?:pXuDr—(kat lecu(YcG﻿zerRV‘Oj:KKB!PW!RArt WhofP))!OyVxsmHxyeR’-Je if;
